# B) Configuration & Change Management
Goal: compare a security baseline (YAML) vs an actual config (nginx-like text) and produce a drift report.

- Read YAML (baseline)
- Inspect a config file (as text)
- Export drift results to CSV

In [6]:
from pathlib import Path
import re, pandas as pd

BASE = Path('..').resolve()
print('Base path:', BASE)

Base path: C:\Project\Miss Shoes\01_Case\B_Config_ChangeMgmt


In [18]:
import json

with open(BASE/'config'/'baseline.json') as f:
    baseline = json.load(f)
with open(BASE/'config'/'actual.conf') as f:
    actual = f.read()
print('Baseline keys:', list(baseline.keys()))
print('Actual (snippet):\n', actual[:200])

Baseline keys: ['tls_min_version', 'hsts', 'server_tokens', 'content_security_policy', 'rate_limit', 'logging']
Actual (snippet):
 ssl_protocols TLSv1 TLSv1.1 TLSv1.2;
# missing HSTS
server_tokens on;
add_header Content-Security-Policy "default-src 'self' 'unsafe-inline'";
# limit_req disabled
access_log on;



In [19]:
# Define simple checks
import re

def has(pattern, text):
    return re.search(pattern, text, re.I|re.M) is not None

checks = [
    ('C-TLS','TLS >= 1.2','High', lambda t: not has(r"ssl_protocols.*TLSv1(\s|;)|TLSv1\.1", t), 'Disable TLSv1/1.1; enforce TLSv1.2+'),
    ('C-HSTS','HSTS enabled','Medium', lambda t: has(r"Strict-Transport-Security", t) or (baseline.get('hsts',{}).get('enabled',False) and baseline['hsts'].get('max_age',0)>=31536000), 'Add HSTS with max-age>=31536000'),
    ('C-SERVER-TOKENS','server_tokens off','Low', lambda t: has(r"server_tokens\\s+off", t) or baseline.get('server_tokens','off')=='off', 'Set server_tokens off'),
    ('C-CSP','CSP restrictive','Medium', lambda t: has(r"Content-Security-Policy\\s+\"default-src 'self'", t), "Use CSP 'default-src self', remove 'unsafe-inline'"),
    ('C-RATE','Rate limiting enabled','Medium', lambda t: has(r"limit_req", t) or 'rate_limit' in baseline, 'Enable rate limiting (e.g., 100r/m)'),
    ('C-LOG','Access logging enabled','Low', lambda t: has(r"access_log\\s+on", t) or baseline.get('logging',{}).get('enabled',False), 'Enable access_log and centralize logs'),
]

rows = []
for cid, desc, sev, fn, rem in checks:
    ok = fn(actual)
    rows.append({'control_id':cid,'description':desc,'severity':sev,'status':'OK' if ok else 'DRIFT','remediation':rem})

import pandas as pd
drift = pd.DataFrame(rows)
drift

,control_id,description,severity,status,remediation
0,C-TLS,TLS >= 1.2,High,DRIFT,Disable TLSv1/1.1; enforce TLSv1.2+
1,C-HSTS,HSTS enabled,Medium,OK,Add HSTS with max-age>=31536000
2,C-SERVER-TOKENS,server_tokens off,Low,OK,Set server_tokens off
3,C-CSP,CSP restrictive,Medium,DRIFT,"Use CSP 'default-src self', remove 'unsafe-inl..."
4,C-RATE,Rate limiting enabled,Medium,OK,"Enable rate limiting (e.g., 100r/m)"
5,C-LOG,Access logging enabled,Low,OK,Enable access_log and centralize logs


In [20]:
# Save as evidence
out_csv = BASE/'outputs'/'config_drift_evidence.csv'
drift.to_csv(out_csv, index=False)
out_csv

WindowsPath('C:/Project/Miss Shoes/01_Case/B_Config_ChangeMgmt/outputs/config_drift_evidence.csv')

In [21]:
# Counts by status/severity for report
summary = drift.groupby(['status','severity']).size().reset_index(name='count')
summary

,status,severity,count
0,DRIFT,High,1
1,DRIFT,Medium,1
2,OK,Low,2
3,OK,Medium,2
